In [1]:
from dask.distributed import Client, WorkerPlugin
from psutil import virtual_memory as mem
import bucket_manager.bucket_manager as bm
import sys

In [2]:
n_workers=1
threads_per_worker=1
mem_per_worker=mem().total//n_workers//2
client = Client(n_workers=n_workers,threads_per_worker=threads_per_worker,memory_limit=mem_per_worker)

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 3.72 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37247,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 3.72 GiB
Comm: tcp://127.0.0.1:33445,Total threads: 1
Dashboard: http://127.0.0.1:41851/status,Memory: 3.72 GiB
Nanny: tcp://127.0.0.1:42775,


In [4]:
import os
import json
kp = '~/lsst_keys.json'
with open(os.path.expanduser(kp), 'r') as kpf:
    kd = json.load(kpf)
    access_key = kd['access_key']
    secret_key = kd['secret_key']
s3_host = 'echo.stfc.ac.uk'

In [5]:
s3_host = 'echo.stfc.ac.uk'
# try:
#     keys = bm.get_keys()
# except KeyError as e:
#     print(f'KeyError {e}', file=sys.stderr)
#     sys.exit()
# access_key = keys['access_key']
# secret_key = keys['secret_key']

s3 = bm.get_resource(access_key, secret_key, s3_host)
bucket_list = bm.bucket_list(s3)

/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'echo.stfc.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
bucket_list

['DRP',
 'LSST-IR-FUSION',
 'LSST-IR-FUSION-Butlers',
 'LSST-IR-FUSION-Butlers-del',
 'LSST-IR-FUSION-TESTSTRATEGY',
 'LSST-IR-FUSION-rdsip005',
 'LSST-IR-FUSION-test',
 'LSST-IR-FUSION-testtree',
 'LSST-IR-FUSION-testtreec',
 'LSST-IR-FUSION-testtreecs',
 'LSST-IR-FUSION_gen3_conversion',
 'dmu4',
 'lsst-dac',
 'lsst-drp-config',
 'lsst-test',
 'lsst-test-3',
 'lsst-test2']

In [7]:
bucket_name = 'LSST-IR-FUSION-testplugin'

In [8]:
class S3WorkerPlugin(WorkerPlugin):
    def __init__(self, access_key, secret_key, s3_host):
        self.access_key = access_key
        self.secret_key = secret_key
        self.s3_host = s3_host
        # self.s3 = bm.get_resource(self.access_key, 
        #                       self.secret_key, 
        #                       self.s3_host)

    def setup(self, worker):
        self.worker = worker
        self.s3 = bm.get_resource(self.access_key, 
                              self.secret_key, 
                              self.s3_host)

def upload_to_s3(file_path, bucket_name, key):
    s3.upload_file(file_path, bucket_name, key)

def list_buckets():
    return bm.bucket_list(s3)



In [9]:
# Register the plugin with the Dask client
client.register_plugin(S3WorkerPlugin(access_key,secret_key,s3_host))



{'tcp://127.0.0.1:33445': {'status': 'OK'}}

In [10]:
future = client.submit(list_buckets)

2024-11-12 09:52:20,848 - distributed.protocol.pickle - ERROR - Failed to serialize <ToPickle: HighLevelGraph with 1 layers.
 0. 140160941370112
>.
Traceback (most recent call last):
  File "/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/distributed/protocol/pickle.py", line 77, in dumps
    result = cloudpickle.dumps(x, **dump_kwargs)
  File "/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/cloudpickle/cloudpickle.py", line 1479, in dumps
    cp.dump(obj)
  File "/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/cloudpickle/cloudpickle.py", line 1245, in dump
    return super().dump(obj)
TypeError: cannot pickle '_thread.lock' object

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/distributed/protocol/pickle.py", line 81, in dumps
    result = cloudpickle.dumps(x, **dump_kwargs)
  File "/home/dave/miniconda3/envs

TypeError: ('Could not serialize object of type HighLevelGraph', '<ToPickle: HighLevelGraph with 1 layers.\n<dask.highlevelgraph.HighLevelGraph object at 0x7f7a2b326c20>\n 0. 140160941370112\n>')

In [11]:
# client.close()

In [12]:
with open(os.path.expanduser('~/lsst-swift-credentials.json'), 'r') as swiftf:
    swiftk = json.load(swiftf)
    user = swiftk['user']
    secret_key = swiftk['secret_key']
user

'lsst:swift'

In [13]:
# conn = bm.get_conn_swift(user, secret_key, s3_host)

In [14]:
# conn

In [23]:
import swiftclient
def get_conn(username, access_key, host):
	return swiftclient.Connection(
        authurl = host,
		user = username,
		key = access_key,
        insecure=True
	)
conn = get_conn(user, secret_key, 'https://s3.echo.stfc.ac.uk/auth/1.0')
def list_containers(conn):
    buckets = []
    for container in conn.get_account()[1]:
        buckets.append(container['name'])
    return buckets

In [24]:
f = client.submit(list_containers, conn)

/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.echo.stfc.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/dave/miniconda3/envs/lsst-uk/lib/python3.10/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.echo.stfc.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [25]:
f.result()

['DRP',
 'LSST-IR-FUSION',
 'LSST-IR-FUSION-Butlers',
 'LSST-IR-FUSION-Butlers-del',
 'LSST-IR-FUSION-TESTSTRATEGY',
 'LSST-IR-FUSION-rdsip005',
 'LSST-IR-FUSION-test',
 'LSST-IR-FUSION-testtree',
 'LSST-IR-FUSION-testtreec',
 'LSST-IR-FUSION-testtreecs',
 'LSST-IR-FUSION_gen3_conversion',
 'dmu4',
 'lsst-dac',
 'lsst-drp-config',
 'lsst-test',
 'lsst-test-3',
 'lsst-test2']

In [21]:
buckets = client.gather(f)

In [22]:
buckets